# Test des données récupérées par API

Ce fichier prend en compte les requêtes pour l'appel API

<b>[Procédures collectives dans la France entière](https://open.urssaf.fr/explore/dataset/procedures-collectives-france-entiere/table/?sort=-dernier_jour_du_trimestre)</b>

Source : URSSAF <br>
Les procédures collectives sont dénombrées par entreprise. <br>
L’Urssaf est à l’origine de ces procédures dans moins d’un cas sur trois.<br>
Les données sont brutes et corrigées des variations saisonnières.

Date :  10/01/2024<br>
Éditeur : <b>Laurent REYNAUD</b>

In [1]:
import requests
import json
import polars as pl

### URL API :

In [2]:
url_api = "https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/procedures-collectives-france-entiere/records?limit=20"

### Requête SQL avec SELECT :

annee, trimestre, nombre_de_procedures_cvs

### Requête SQL avec WHERE :

nature_de_procedure LIKE 'Liquidation judiciaire' AND annee > date'2012'

### Requête SQL avec GROUP_BY :

### Requête SQL avec ORDER BY :

annee ASC, trimestre ASC

### Requête SQL avec LIMIT :

100

### Variables à saisir avec les requêtes SQL WHERE / LIMIT... ci-avant :

In [2]:
procedure=['Liquidation judiciaire', 'Redressement judiciaire', 'Sauvegarde']

### URL

[Encodage-pourcent](https://fr.wikipedia.org/wiki/Encodage-pourcent)

In [3]:
# Accès à l'URL avec le requêtage appliqué : 
url = f"https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/procedures-collectives-france-entiere/records?select=annee%2C%20trimestre%2C%20nombre_de_procedures_cvs&where=nature_de_procedure%20LIKE%20%27{procedure[1]}%27%20AND%20annee%20%3E%20date%272012%27&order_by=annee%20ASC%2C%20trimestre%20ASC&limit=100"

In [4]:
# Requête url
response = requests.get(url)

In [5]:
# Code statut html
response.status_code

200

In [6]:
# Récupération du contenu au format json
data = json.loads(response.content)

In [23]:
# Affichage des données
df = pl.DataFrame(data['results'])

In [24]:
# Renommage des champs
df = df.rename({'annee': 'Annee', 'trimestre': 'Trimestre',
                'nombre_de_procedures_cvs' : "Nombre de procédures"})

In [27]:
# Concaténation des champs Annee et Trimestre
df.with_columns(pl.concat_str([pl.col('Annee'), 
                               pl.lit(' - '), 
                               pl.col('Trimestre')]).alias('Periode'))

Annee,Trimestre,Nombre de procédures,Periode
str,i64,i64,str
"""2013""",1,4003,"""2013 - 1"""
"""2013""",2,4146,"""2013 - 2"""
"""2013""",3,4170,"""2013 - 3"""
"""2013""",4,4064,"""2013 - 4"""
"""2014""",1,4034,"""2014 - 1"""
"""2014""",2,3937,"""2014 - 2"""
"""2014""",3,3893,"""2014 - 3"""
"""2014""",4,3866,"""2014 - 4"""
"""2015""",1,4232,"""2015 - 1"""
